# PyVacation


In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings 

# Suppress warnings
warnings.filterwarnings('ignore')   

# Import API key
from api_keys import geoapify_key

In [4]:
# Create a Pandas DataFrame with the CSV file created in the WeatherPy script
city_data_df = pd.read_csv("../output_data/cities.csv")

# Preview the DataFrame 
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapuskasing,49.4169,-82.4331,26.05,50,75,3.09,CA,1691532253
1,1,vorgashor,67.5833,63.9500,9.05,75,100,2.97,RU,1691532408
2,2,blackmans bay,-43.0167,147.3167,2.63,94,39,2.98,AU,1691532352
3,3,hermanus,-34.4187,19.2345,11.17,82,0,5.32,ZA,1691532355
4,4,yellowknife,62.4560,-114.3525,20.87,46,3,7.60,CA,1691532524


### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point is the humidity in each city.

In [5]:
# Create a dynamic humidity map 
map_plot = city_data_df.hvplot.points(
    "Lng", "Lat", geo=True,
    frame_height=400, frame_width=800,  
    hover_cols=["City"], tiles="OSM",
    color="City", size="Humidity")
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria for vacation
filtered_cities_df = city_data_df[
    (city_data_df['Max Temp'] >= 18) & (city_data_df['Max Temp'] <= 26) &
    (city_data_df['Humidity'] >= 30) & (city_data_df['Humidity'] <= 70) &
    (city_data_df['Cloudiness'] <= 45) & (city_data_df['Lat'] >= -10) & 
    (city_data_df['Lat'] <= 45) &
    (city_data_df['Wind Speed'] <= 25) &  (city_data_df['Lng'] >= -150) & (city_data_df['Lng'] <= 80) 
]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Preview the filtered data
filtered_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
143,143,lompoc,34.6391,-120.4579,24.88,69,10,2.68,US,1691532350
220,220,jamestown,42.0970,-79.2353,23.90,68,0,8.75,US,1691532385
240,240,ciutadella,40.0011,3.8414,24.45,64,0,3.10,ES,1691532630
257,257,bunyan,38.8463,35.8603,25.12,49,1,1.25,TR,1691532639
269,269,hashtrud,37.4779,47.0508,20.47,49,0,1.34,IR,1691532644
270,270,pimentel,-6.8367,-79.9342,24.16,69,40,7.72,PE,1691532645
284,284,mugia,43.1041,-9.2179,20.51,70,0,2.60,ES,1691532651
297,297,smara,26.7384,-11.6719,23.63,64,0,7.72,EH,1691532658
397,397,chadron,42.8294,-102.9999,24.82,57,0,4.63,US,1691532706
409,409,xuddur,4.1213,43.8894,21.86,69,41,8.69,SO,1691532711


### Step 3: Create a new DataFrame called `hotel_df`

In [7]:
# Use the Pandas copy function to create hotel_df DataFrame
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column "Hotel Name" to the DataFrame
hotel_df['Hotel Name'] = ''

# Preview the DataFrame
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
143,lompoc,US,34.6391,-120.4579,69,
220,jamestown,US,42.0970,-79.2353,68,
240,ciutadella,ES,40.0011,3.8414,64,
257,bunyan,TR,38.8463,35.8603,49,
269,hashtrud,IR,37.4779,47.0508,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000 
api_key = "geoapify_key"  

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Set the full endpoint URL with parameters
    endpoint_url = f"https://api.geoapify.com/v2/places?categories=accommodation.hotel&radius={radius}&lat={lat}&lon={lng}&apiKey={geoapify_key}"

    # Make an API request using the full endpoint URL
    response = requests.get(endpoint_url)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

Starting hotel search
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
jamestown - nearest hotel: DoubleTree Jamestown
ciutadella - nearest hotel: El Claustre de Ciutadella
bunyan - nearest hotel: Kayseri Loft Hotel
hashtrud - nearest hotel: No hotel found
pimentel - nearest hotel: Hospedaje Calle del Sol
mugia - nearest hotel: Solpor do Coído
smara - nearest hotel: Hôtel Amine فندق الأمين
chadron - nearest hotel: Holiday Inn Express & Suites Chadron
xuddur - nearest hotel: Shabele Hotel
ammon - nearest hotel: La Quinta Inn & Suites
tzimol - nearest hotel: Cabanas de El Chiflon
tumbi - nearest hotel: Mwafrica hotel
osh - nearest hotel: Rayan Hotel
vedene - nearest hotel: Novotel Avignon Nord
malanje - nearest hotel: Hotel PORTVGALIA
kananga - nearest hotel: Hôtel Résidence


In [9]:
# Preview the dataframe
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
143,lompoc,US,34.6391,-120.4579,69,Embassy Suites by Hilton Lompoc Central Coast
220,jamestown,US,42.0970,-79.2353,68,DoubleTree Jamestown
240,ciutadella,ES,40.0011,3.8414,64,El Claustre de Ciutadella
257,bunyan,TR,38.8463,35.8603,49,Kayseri Loft Hotel
269,hashtrud,IR,37.4779,47.0508,49,No hotel found
270,pimentel,PE,-6.8367,-79.9342,69,Hospedaje Calle del Sol
284,mugia,ES,43.1041,-9.2179,70,Solpor do Coído
297,smara,EH,26.7384,-11.6719,64,Hôtel Amine فندق الأمين
397,chadron,US,42.8294,-102.9999,57,Holiday Inn Express & Suites Chadron
409,xuddur,SO,4.1213,43.8894,69,Shabele Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
# Filter out rows with "No hotel found"
filtered_hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]

# Create the hotel plot with points using hvplot
hotel_plot = filtered_hotel_df.hvplot.points(
    "Lng", "Lat", geo=True,
    frame_height=400, frame_width=800,
    hover_cols=["Country", "City", "Hotel Name"],
    tiles="OSM",
    color="Hotel Name",
)
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country,City)